<a href="https://colab.research.google.com/github/data-IA-2022/FAO_Project_Tarik_FengFeng/blob/main/Copie_de_FAO_project_Fengfeng_Tarik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><center>Data Analyse au sein de la FAO (Food and Agriculture Organization</h1>

<h2>Contexte du projet</h2>

Au sein du service Data Analytics and Prospective de la FAO, vous êtes en charge d'analyser les données produites par les Nations Unies.

Ces analyses conduisent à des rapports de synthèse à destination des décideurs.

Pour êtes également en charge de la partie Prospective, c'est à dire donner des tendances et des évolutions probables quant à la production agricole, à l'alimentation et la nutrition mondiales.

In [83]:
# Copie des fichiers csv du repertoire github et suppréssions des ancien en cas de mise à jour

!rm -r /content/FAO_Project_Tarik_FengFeng
!git clone https://github.com/data-IA-2022/FAO_Project_Tarik_FengFeng.git

Cloning into 'FAO_Project_Tarik_FengFeng'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 73 (delta 37), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (73/73), done.


In [84]:
#importation des librairies python
import os
import pandas as pd
import glob

In [85]:
# Récupération des chemin vers les fichiers csv
csv_path = '/content/FAO_Project_Tarik_FengFeng/fao_2013'
def find_csv_filenames(path_to_dir=os.getcwd(), extension="csv"):
    filenames = []
    for name in glob.glob(f'{path_to_dir}/*.{extension}'):
      filenames.append(name)
    return filenames
all_csv_files= find_csv_filenames(path_to_dir=csv_path)
print(all_csv_files)

['/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_cereal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_population.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_vegetal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_animal.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/pays-continents.csv', '/content/FAO_Project_Tarik_FengFeng/fao_2013/FAOSTAT_2013_sous_alimentation.csv']


In [86]:
# Création d'un dictionaire avec nom des fichiers csv comme Key et dataframe comme value

col_names= ['Code Domaine', 'Domaine', 'Code Pays', 'Pays', 'Code Élément',
       'Élément', 'Code Produit', 'Produit', 'Code année', 'Année', 'Unité',
       'Valeur', 'Symbole', 'Description du Symbole']
df_dict = {}
for f in all_csv_files:
  df_name= f.split("/")[-1].replace(".csv", "").split("_")[-1]
  if "FAOSTAT_2013" in f:
    df_dict[f'{df_name}'] = pd.read_csv(f, encoding='utf-8', quotechar='"', delimiter=',', names= col_names, header=0)
  else:
    df_dict[f'{df_name}'] = pd.read_csv(f, encoding='utf-8', quotechar='"', delimiter=',')

print(df_dict.keys())

dict_keys(['cereal', 'population', 'vegetal', 'animal', 'pays-continents', 'alimentation'])


In [88]:
# Définition des Dataframes

animal= df_dict["animal"]
cereal= df_dict["cereal"]
population= df_dict["population"]
alimentation= df_dict["alimentation"]
vegetal= df_dict["vegetal"]
pays_continents= df_dict["pays-continents"]

In [89]:
# Définir les codes pays comme index des Df
animal.index= animal["Code Pays"]
animal.index.name= "Code_Pays_id"
cereal.index= cereal["Code Pays"]
cereal.index.name= "Code_Pays_id"
population.index= population["Code Pays"]
population.index.name= "Code_Pays_id"
alimentation.index= alimentation["Code Pays"]
alimentation.index.name= "Code_Pays_id"
vegetal.index= vegetal["Code Pays"]
vegetal.index.name= "Code_Pays_id"
pays_continents.index= pays_continents["Code Pays"]
pays_continents.index.name= "Code_Pays_id"

# Nouvelle section

In [ ]:
# nb lignes et nb colonnes de nos df
print(animal.shape)
print(cereal.shape)
print(population.shape)
print(alimentation.shape)
print(vegetal.shape)
print(pays_continents.shape)

In [ ]:
# Join population et cereal production et pays continents

df_pop_cerealprod = population.join(cereal, on= "Code Pays",how="left", lsuffix="_pop", rsuffix="_cer" )
df_pop_cerealprod = df_pop_cerealprod.join(pays_continents, on= "Code Pays",how="left", lsuffix="_popcer", rsuffix="_conti" )
df_pop_cerealprod = df_pop_cerealprod.query("Produit_cer in ('Blé') and Élément_cer in ('Production')")
df_pop_cerealprod = df_pop_cerealprod[["Valeur_pop","Valeur_cer","Pays_pop","Groupe de pays"]]
print(df_pop_cerealprod)

In [92]:
# Graphiques scatter plot de la production de blé par rapport à la population par pays

# Error divide by zero encountered in log prendre les valeurs supérieur


# Avec plotly
import plotly.express as px
import plotly.io as io
io.renderers.default='colab'
df = df_pop_cerealprod
fig = px.scatter(df, x="Valeur_pop", y="Valeur_cer", color="Groupe de pays", log_x=True, log_y=True)
fig.show()